In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter

In [49]:
start_date = '2023-12-18'
# end_date ='2021-08-28'

start_date = pd.to_datetime(start_date) 
# end_date = pd.to_datetime(end_date)
end_date = start_date + pd.Timedelta(days=35)


In [50]:
target = '../Chanae-combined_files'
# sunny rainy winter
output_dir = '../Chanae-Graph/2023/winter'

In [51]:
# sixfactor_dir = ['d2m','RH','temperature','rainfall','ws','uvb','durationOfDay','sevenFactor']
sixfactor_dir = ['sevenFactor']
for dir in sixfactor_dir:
    if not os.path.exists(os.path.join(output_dir,dir)):
        os.makedirs(os.path.join(output_dir,dir))

In [52]:
colors = {
    'd2m': 'deeppink',
    'RH': 'green',
    'minTemp': 'orange',
    'meanTemp': 'orangered',
    'maxTemp': 'darkred',
    'tp': 'blue',
    'WS': 'dodgerblue',
    'uvb': 'darkviolet',
    'duration_of_day': 'black',
}

for file in os.listdir(target):
    df = pd.read_excel(os.path.join(target, file))
    df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')    
    filtered_data = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    df = filtered_data

    # print(df.head(7))    
    # แปลงข้อมูลในคอลัมน์ 'date' เป็นชนิดข้อมูล datetime
    df['date'] = pd.to_datetime(df['date'])

    # กำหนด 'date' เป็น index
    df.set_index('date', inplace=True)

    # คำนวณระยะเวลาของแต่ละวัน
    df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])

    # สร้าง DataFrame ใหม่โดยรวมข้อมูลทุก 7 วัน
    df_resampled = df.resample('7D').agg({'d2m': 'mean', 
                                          'RH': 'mean', 
                                          'minTemp': 'min',
                                          'maxTemp': 'max',
                                          'meanTemp': 'mean',
                                          'tp': 'sum',
                                          'WS': 'mean',
                                          'uvb': 'mean',
                                          'duration_of_day': 'mean'
                                          })

    # เลื่อน index ไป 7 วัน
    df_resampled.index = df_resampled.index + pd.DateOffset(days=6)

    # # ตัดข้อมูลเฉพาะที่ต้องการแสดง
    end_date = df.index[-1]  # วันที่สิ้นสุดของข้อมูล
    df_resampled = df_resampled[df_resampled.index <= end_date]

    # แปลงระยะเวลาจากวินาทีเป็น ชั่วโมงและนาที
    df_resampled['duration_hours'] = df_resampled['duration_of_day'].dt.seconds // 3600
    df_resampled['duration_minutes'] = (df_resampled['duration_of_day'].dt.seconds % 3600) // 60

    # # พล็อตกราฟ
    plt.figure(figsize=(10, 6))

    # พล็อตกราฟจุดน้ำค้าง
    plt.plot(df_resampled.index, df_resampled['d2m'], label='Dew Point (°C)', marker='s', color=colors['d2m'])
    plt.plot(df_resampled.index, df_resampled['RH']*100, label='RH (%)', marker='o', color=colors['RH'])
    plt.plot(df_resampled.index, df_resampled['minTemp'], label='minTemp (°C)', marker='s', color=colors['minTemp'])
    plt.plot(df_resampled.index, df_resampled['meanTemp'], label='meanTemp (°C)', marker='s', color=colors['meanTemp'])
    plt.plot(df_resampled.index, df_resampled['maxTemp'], label='maxTemp (°C)', marker='s', color=colors['maxTemp'])
    plt.plot(df_resampled.index, df_resampled['tp']*1000, label='Rainfall (mm)', marker='o', color=colors['tp'])
    plt.plot(df_resampled.index, df_resampled['WS'], label='WS (m/s)', marker='o', color=colors['WS'])
    # 0 -> /1000
    plt.plot(df_resampled.index, df_resampled['uvb']/1000, label='radiation (kJ/m^2) ', marker='o', color=colors['uvb'])
    plt.plot(df_resampled.index, df_resampled['duration_hours'] + df_resampled['duration_minutes'] / 60, label='duration_of_day (hours)', marker='o', color=colors['duration_of_day'])

    # ปรับตำแหน่ง x ของ marker ให้ตรงกับข้อมูลที่ถูกรวมรวมแล้ว
    plt.xticks(df_resampled.index)


    # ปรับแต่งกราฟ
    plt.title('Seven Factor (7-day Interval)')
    plt.xlabel('Date')
    # ปรับตำแหน่ง label ให้ไม่บังกราฟ
    # plt.legend()
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.xticks(rotation=45)
    # plt.tight_layout()
    plt.grid(True)

    # บันทึกภาพเป็นไฟล์ PNG
    output_file = os.path.join(output_dir,sixfactor_dir[0], f'SevenFactor_{file.split(".")[0]}.png')
    plt.savefig(output_file, bbox_inches='tight')
    plt.close()
    
    print(f"Saved graph to {output_file}")


C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])


Saved graph to ../Chanae-Graph/2023/winter\sevenFactor\SevenFactor_Chanae.png


C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])


Saved graph to ../Chanae-Graph/2023/winter\sevenFactor\SevenFactor_Chang-Phueak.png


C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])


Saved graph to ../Chanae-Graph/2023/winter\sevenFactor\SevenFactor_Du-Song-Yo.png


C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_21572\1248333778.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['duration_of_day'] = pd.to_datetime(df['sunset']) - pd.to_datetime(df['sunrise'])


Saved graph to ../Chanae-Graph/2023/winter\sevenFactor\SevenFactor_Phadung-Mat.png
